<a href="https://colab.research.google.com/github/DataScienceAndEngineering/deep-learning-final-project-project-sidewalk/blob/rabiul/notebooks%20/Rabiul/Baseline_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Loading Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
import os
import seaborn as sns
import pandas as pd
from skimage.filters import roberts, sobel, scharr, prewitt
from scipy import ndimage as nd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from google.colab import drive
import pickle
from sklearn import metrics

In [ ]:
#loading data and model 
!gdown --id 1-3gbEPrVnD0YZ38ZixZrRjo3y2-NeOgr -O test.csv
!gdown --id 1TRDaaX13CqHE8xzIXKPnhLEhniWlUR9a -O train.csv
!gdown --id 1-14yL80KGLJzF2D6j3BXTH1dLa2knz_W -O model.sav

In [ ]:
model = pickle.load(open('/content/model.sav', 'rb'))

In [ ]:
X_train, X_test, y_train, y_test = train_test_data('/content/train.csv', '/content/test.csv')
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
print(rf_accuracy(model, X_test,y_test))

In [ ]:
print(rf_accuracy(model, X_train,y_train))

# Data

###  Loading Data

In [ ]:
%%time
#downloading preprocessed data
!gdown --id 12rNfOiM4allOn8t4l0ErpvgmA9hEZPh2 -O processed_labels.zip
!gdown --id 145Ul0rdbQcX98lQz8IO5eOtI0ZlvKwz6 -O processed_images.zip

In [ ]:
%%time
#unzipping the preprocessed data to local dir
!unzip /content/processed_images.zip
!unzip /content/processed_labels.zip

### Pre-Processing Data Step 1

In [ ]:
#path to the directory for both image and label data 
img_path = '/content/content/processed_images/*'
lab_path = '/content/content/processed_labels/*'
#one sample data point
#/content/content/processed_images/aachen_000000_000019.png
#/content/content/processed_labels/aachen_000000_000019.png

In [ ]:
#loading the data indo a list, numpy array
%%time
train_images = glob.glob(img_path)
train_labels = glob.glob(lab_path)
print(len(train_images),len(train_labels))


In [ ]:
n=40
m=2310

In [ ]:
t = np.array(x_train['gray'].values)
t = t.reshape(int(t.shape[0]/40000),40000)
display_img(t[0].reshape(200,200))

In [ ]:
#test and train images 
x_train = create_train_image_df(train_images[:n])
x_test = create_test_image_df(train_images[2300:m])
print(x_train.shape,x_test.shape )
x_train.head()

In [ ]:
x_test.head(2)

In [ ]:
#Creating train mask image Data Frame
%%time
dim = (200,200)
y_train = pd.DataFrame()

for label_dir in train_labels[:n]:
  df2 = pd.DataFrame()
  lab = cv2.imread(label_dir, 0)
  #lab = cv2.resize(lab, dim).reshape(-1)
  df2['label'] = cv2.resize(lab, dim).reshape(-1) 
  y_train = pd.concat([y_train, df2])
print(y_train.shape)
y_train.head(2)

In [ ]:
#Creating test mask image Data Frame
%%time
dim = (200,200)
y_test = pd.DataFrame()

for label_dir in train_labels[2300:m]:
  df2 = pd.DataFrame()
  lab = cv2.imread(label_dir, 0)
  #lab = cv2.resize(lab, dim).reshape(-1)
  df2['label'] = cv2.resize(lab, dim).reshape(-1) 
  y_test = pd.concat([y_test, df2])
print(y_test.shape)
y_test.head(2)

In [ ]:
df_train = pd.concat([x_train, y_train], axis=1)
print(df_train.shape)
df_train.head(2)

In [ ]:
df_test = pd.concat([x_test, y_test], axis=1)
print(df_test.shape)
df_test.head(2)

In [ ]:
print(df_train.shape)
print(df_test.shape)

In [ ]:
cd /content/drive/MyDrive/DL_Project

In [ ]:
#only execute this cell to save the preprocessed dataframe to google drive 
df_train.to_csv('df_train.csv', index=False)
df_test.to_csv('df_test.csv', index=False)
#Changing directory back to "Content"


In [ ]:
cd /content

## PreProcess Data Step 2

In [ ]:
#print(df_train.shape)
#print(df_test.shape)
print(df_train.shape, df_test.shape)

In [ ]:
#Execute this cell only if you want to use GPU and GPU is setup
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
#Execute this cell only if you want to remove all zero value where label = 0. 
df_train = df_train[df_train.label != 0]
df_test = df_test[df_test.label != 0]
print(df_train.shape, df_test.shape)

In [ ]:
X_Train = df_train.drop(labels = ["label"], axis=1)
Y_Train = df_train["label"].values

X_Test = df_test.drop(labels = ["label"], axis=1)
Y_Test = df_test["label"].values


print((X_Train.shape, Y_Train.shape),(X_Test.shape, Y_Test.shape))

In [ ]:
Y_Train = LabelEncoder().fit_transform(Y_Train)
Y_Test = LabelEncoder().fit_transform(Y_Test)
print(Y_Train.shape, Y_Test.shape)

# MODEL

In [ ]:
model = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=None, 
                            min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                            max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, 
                            bootstrap=True, oob_score=False, n_jobs=None, random_state=2, 
                            verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, 
                            max_samples=None)

## FIT & Save Model

In [ ]:
cd /content/drive/MyDrive/DL_Project

In [ ]:
%%time
## Train the model on training data
model.fit(X_Train, Y_Train)

In [ ]:
#Save the trained model as pickle string to google drive for future use 
filename = "rf_model2_withzero.sav"
pickle.dump(model, open(filename, 'wb'))
#Changing directory back to "Content"
cd /content

## Predict

In [ ]:
prediction_test = model.predict(X_Test)

In [ ]:
print ("Testing Accuracy = ", metrics.accuracy_score(Y_Test, prediction_test))

In [ ]:
Y_pred = prediction_test.reshape(int(prediction_test.shape[0]/40000),40000)
Y_true = Y_Test.reshape(int(Y_Test.shape[0]/40000),40000)
print(Y_true.shape,Y_pred.shape )

In [ ]:
print ("Testing Accuracy = ", metrics.accuracy_score(Y_true[0].reshape(200,200), Y_pred[0].reshape(200,200)))

In [ ]:
import numpy as np
from sklearn.metrics import jaccard_score

In [ ]:
print ("Training Accuracy = ", jaccard_score(Y_true, Y_pred,average='micro'))

# Functions

In [ ]:
def gabor_feature_extraction(img, lamda, gamma):
  kernel = cv2.getGaborKernel((8, 8), 1, 0, lamda, gamma, 0, ktype=cv2.CV_32F)
  gabor_img = cv2.filter2D(img, cv2.CV_8UC3, kernel)
  return gabor_img.reshape(-1,1)/255


In [ ]:
def preprocess_image(img):
  #function will accept a numpy.ndarray

  #GAUSSIAN blur 1
  gblur = cv2.GaussianBlur(img, (5,5),7)
  #Laplacian
  laplacian = cv2.Laplacian(img, cv2.CV_64F, ksize=7) 
  #SOBEL
  sobele = cv2.Sobel(img,cv2.CV_64F, 0,1,ksize=5)
  #CANNY 
  cany = cv2.Canny(np.uint8(sobele), 100,200)

  #return cany
  return (gblur.reshape(-1)/255,laplacian.reshape(-1)/255,sobele.reshape(-1)/255,cany.reshape(-1)/255)

In [ ]:
def preprocess_image_org(img):
  #GAUSSIAN blur 1
  blur1 = nd.gaussian_filter(img, sigma=3)
  #GAUSSIAN blur2
  blur2 = nd.gaussian_filter(img, sigma=7)
  #SOBEL
  sobele = sobel(blur2)
  #CANNY 
  cany = cv2.Canny(np.uint8(sobele), 100,200)
  #return cany
  return (blur1.reshape(-1,1),blur2.reshape(-1,1),sobele.reshape(-1,1),cany.reshape(-1,1))

In [ ]:
def display_img(img):
  fig = plt.figure(figsize=(8,8))
  ax = fig.add_subplot(111)
  ax.imshow(img,cmap='gray')

In [ ]:
def load_img(image_path):
  return cv2.imread(image_path, 0).astype(np.float32)/255


In [ ]:
def generate_one_test_image(image_dir):
  img = cv2.imread(image_dir, 0)
  g1 = img/255
  _,thresh3  = cv2.threshold(img, 80, 255, cv2.THRESH_BINARY_INV)
  binay_inv = thresh3/255
  adaptive_thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11, 11)/255
  gb1 = gabor_feature_extraction(img,.8, .05 )/255
  gb2 = gabor_feature_extraction(img,1.6, .5 )/255
  blur, laplacian, sobel, cany = preprocess_image(img)
  return np.hstack((g1.reshape(-1,1), binay_inv.reshape(-1,1), adaptive_thresh.reshape(-1,1), 
                    gb1.reshape(-1,1), gb2.reshape(-1,1), blur.reshape(-1,1), 
                    laplacian.reshape(-1,1), sobel.reshape(-1,1), cany.reshape(-1,1)))


In [ ]:
def generate_one_test_image_df(image_dir):
  dim = (200,200)
  test_df = pd.DataFrame()
  img = cv2.imread(image_dir, 0)
  img = cv2.resize(img, dim)
  g1 = img/255
  _,thresh3  = cv2.threshold(img, 80, 255, cv2.THRESH_BINARY_INV)
  binay_inv = thresh3/255
  adaptive_thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11, 11)/255
  gb1 = gabor_feature_extraction(img,.8, .05 )/255
  gb2 = gabor_feature_extraction(img,1.6, .5 )/255
  blur, laplacian, sobel, cany = preprocess_image(img)

  test_df['gray']= g1.reshape(-1)
  test_df['binay_inv']= binay_inv.reshape(-1)
  test_df['adaptive_thresh']=adaptive_thresh.reshape(-1)
  test_df['gabor1'] = gabor_feature_extraction(img,.8, .05 ).reshape(-1)
  test_df['gabor2'] = gabor_feature_extraction(img,1.6, .5 ).reshape(-1)
  test_df['blur'], test_df['laplacian'], test_df['sobelx'], test_df['canny'] = blur.reshape(-1), laplacian.reshape(-1), sobel.reshape(-1), cany.reshape(-1)

  return test_df


In [ ]:
def generate_one_test_image_df_a(image_dir):
  #this function is getting an image as an input 
  test_df = pd.DataFrame()
  #img = cv2.imread(image_dir, 0)
  img = image_dir
  g1 = img/255
  _,thresh3  = cv2.threshold(img, 80, 255, cv2.THRESH_BINARY_INV)
  binay_inv = thresh3/255
  adaptive_thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11, 11)/255
  gb1 = gabor_feature_extraction(img,.8, .05 )/255
  gb2 = gabor_feature_extraction(img,1.6, .5 )/255
  blur, laplacian, sobel, cany = preprocess_image(img)

  test_df['gray']= g1.reshape(-1)
  test_df['binay_inv']= binay_inv.reshape(-1)
  test_df['adaptive_thresh']=adaptive_thresh.reshape(-1)
  test_df['gabor1'] = gabor_feature_extraction(img,.8, .05 ).reshape(-1)
  test_df['gabor2'] = gabor_feature_extraction(img,1.6, .5 ).reshape(-1)
  test_df['blur'], test_df['laplacian'], test_df['sobelx'], test_df['canny'] = blur.reshape(-1), laplacian.reshape(-1), sobel.reshape(-1), cany.reshape(-1)

  return test_df

In [ ]:
def visualize_images(img, gt, pred):
    if pred is not None:
        fig, axes = plt.subplots(1, 3, figsize=(12, 8))
    else:
        fig, axes = plt.subplots(1, 2, figsize=(8, 8))

    axes[0].imshow(img)
    axes[0].set_title('Actual Image')

    axes[1].imshow(gt)
    axes[1].set_title('Masked Image')
    
    if pred is not None:
        axes[2].imshow(pred)
        axes[2].set_title('Predicted Image')

In [ ]:
def conv_block(X,filters,block):
    # resiudal block with dilated convolutions
    # add skip connection at last after doing convoluion

    b = 'block_'+str(block)+'_'
    f1,f2,f3 = filters
    X_skip = X

    # block_a
    X = Conv2D(filters=f1,kernel_size=(1,1),dilation_rate=(1,1),
               padding='same',kernel_initializer='he_normal',name=b+'a')(X)
    X = BatchNormalization(name=b+'batch_norm_a')(X)
    X = LeakyReLU(alpha=0.2,name=b+'leakyrelu_a')(X)
    # block_b
    X = Conv2D(filters=f2,kernel_size=(3,3),dilation_rate=(2,2),
               padding='same',kernel_initializer='he_normal',name=b+'b')(X)
    X = BatchNormalization(name=b+'batch_norm_b')(X)
    X = LeakyReLU(alpha=0.2,name=b+'leakyrelu_b')(X)
    # block_c
    X = Conv2D(filters=f3,kernel_size=(1,1),dilation_rate=(1,1),
               padding='same',kernel_initializer='he_normal',name=b+'c')(X)
    X = BatchNormalization(name=b+'batch_norm_c')(X)
    # skip_conv
    X_skip = Conv2D(filters=f3,kernel_size=(3,3),padding='same',name=b+'skip_conv')(X_skip)
    X_skip = BatchNormalization(name=b+'batch_norm_skip_conv')(X_skip)
    # block_c + skip_conv
    X = Add(name=b+'add')([X,X_skip])
    X = ReLU(name=b+'relu')(X)
    return X
    
def base_feature_maps(input_layer):
    # base covolution module to get input image feature maps 
    
    # block_1
    base = conv_block(input_layer,[16,16,32],'1')
    # block_2
    base = conv_block(base,[16,16,32],'2')
    return base

def pyramid_feature_maps(input_layer):
    # pyramid pooling module
    
    base = base_feature_maps(input_layer)
    # red
    red = GlobalAveragePooling2D(name='red_pool')(base)
    red = tf.keras.layers.Reshape((1,1,32))(red)
    red = Conv2D(filters=32,kernel_size=(1,1),name='red_1_by_1')(red)
    red = UpSampling2D(size=128,interpolation='bilinear',name='red_upsampling')(red)
    red = tf.image.resize(red, [IMG_SIZE, IMG_SIZE])
    # yellow
    yellow = AveragePooling2D(pool_size=(2,2),name='yellow_pool')(base)
    yellow = Conv2D(filters=32,kernel_size=(1,1),name='yellow_1_by_1')(yellow)
    yellow = UpSampling2D(size=2,interpolation='bilinear',name='yellow_upsampling')(yellow)
    yellow = tf.image.resize(yellow, [IMG_SIZE, IMG_SIZE])
    # blue
    blue = AveragePooling2D(pool_size=(4,4),name='blue_pool')(base)
    blue = Conv2D(filters=32,kernel_size=(1,1),name='blue_1_by_1')(blue)
    blue = UpSampling2D(size=4,interpolation='bilinear',name='blue_upsampling')(blue)
    blue = tf.image.resize(blue, [IMG_SIZE, IMG_SIZE])
    # green
    green = AveragePooling2D(pool_size=(8,8),name='green_pool')(base)
    green = Conv2D(filters=32,kernel_size=(1,1),name='green_1_by_1')(green)
    green = UpSampling2D(size=8,interpolation='bilinear',name='green_upsampling')(green)
    green = tf.image.resize(green, [IMG_SIZE, IMG_SIZE])
    # base + red + yellow + blue + green
    return tf.keras.layers.concatenate([base,red,yellow,blue,green])

def last_conv_module(input_layer):
    X = pyramid_feature_maps(input_layer)
    X = Conv2D(filters=3,kernel_size=3,padding='same',name='last_conv_3_by_3')(X)
    X = BatchNormalization(name='last_conv_3_by_3_batch_norm')(X)
    X = Activation('sigmoid',name='last_conv_relu')(X)
    return X

In [ ]:
def plot(history):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(1,len(acc)+1)

  plt.title('Training and validation accuracy')
  plt.plot(epochs, acc, color='blue', label='Train')
  plt.plot(epochs, val_acc, color='orange', label='Val')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.legend()

  _ = plt.figure()
  plt.title('Training and validation loss')
  plt.plot(epochs, loss, color='blue', label='Train')
  plt.plot(epochs, val_loss, color='orange', label='Val')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()

In [ ]:
def create_test_image_df(train_images,n):
  '''provide a list of image directory and n= number of image'''
  image_df = pd.DataFrame()
  for img_dir in train_images[:n]:
    image_df = pd.concat([image_df, generate_one_test_image_df(img_dir)])
  return image_df

In [ ]:
def create_train_image_df(train_images):
  '''provide a list of image directory and n= number of image you want , test image starts from 2300'''
  image_df = pd.DataFrame()
  for img_dir in train_images:
    image_df = pd.concat([image_df, generate_one_test_image_df(img_dir)])
  print('train', image_df.shape)
  return image_df

def create_test_image_df(train_images):
  import pandas as pd
  '''provide a list of image directory and n= number of image'''
  image_df = pd.DataFrame()
  for img_dir in train_images:
    single_img_df = generate_one_test_image_df(img_dir)
    image_df = pd.concat([image_df,single_img_df ])
  print('test', image_df.shape)
  return image_df

In [ ]:
def train_test_data(df_train, df_test):
  '''df_train, df_test > provide both test and train df with image data and label data'''
  df_train = pd.read_csv(df_train) 
  df_test = pd.read_csv(df_test) 
  X_Train = df_train.drop(labels = ["label"], axis=1)
  Y_Train = df_train["label"].values

  X_Test = df_test.drop(labels = ["label"], axis=1)
  Y_Test = df_test["label"].values

  Y_Train = LabelEncoder().fit_transform(Y_Train)
  Y_Test = LabelEncoder().fit_transform(Y_Test)

  return X_Train,X_Test, Y_Train, Y_Test

In [ ]:
def rf_accuracy(model, X, Y):
  'provide the randomforest model, X: test/train data, y: test/train label '
  prediction_test = model.predict(X)
  Y_pred = prediction_test.reshape(int(prediction_test.shape[0]/40000),40000)
  Y_true = Y.reshape(int(Y.shape[0]/40000),40000)
  return metrics.accuracy_score(Y_true[0].reshape(200,200), Y_pred[0].reshape(200,200))

# WIP

In [ ]:
train_images = np.array(train_img)
train_labels = np.array(train_lab)
print(train_images.shape, train_labels.shape)

In [ ]:
#test_img = cv2.imread('/content/content/processed_images/aachen_000000_000019.png', 0)
#test_lab = cv2.imread('/content/content/processed_labels/aachen_000000_000019.png',0)

test_img = train_images[210].reshape(dim)
test_lab= train_labels[210].reshape(dim)
train_img = train_images[68].reshape(dim)
train_lab= train_labels[68].reshape(dim)

In [ ]:
display_img(test_img)

In [ ]:
display_img(test_lab)

In [ ]:
#fX = generate_one_test_image_df(train_images[201])
fX_test = generate_one_test_image_df_a(test_img)
fX_test.shape

In [ ]:
#fX = generate_one_test_image_df(train_images[201])
fX_train = generate_one_test_image_df_a(train_img)
fX_train.shape

In [ ]:
print((X_Train.shape, Y_Train.shape),(X_Test.shape, Y_Test.shape))
#prediction_test = model.predict(fX)

## 2

In [ ]:
train_img = []
dim = (200,200)
for img in train_images:
  m = cv2.imread(img, 0)
  train_img.append(cv2.resize(m, dim).reshape(-1))
print(len(train_img), train_img[0].shape)

In [ ]:
display_img(train_img[0].reshape(dim))

In [ ]:
train_images = np.array(train_img)

In [ ]:
type(train_img), type(train_img[0]),train_img[0].shape

In [ ]:
type(train_images), type(train_images[0]),train_images[0].shape

In [ ]:
train_lab = []
dim = (200,200)
for img in train_labels:
  m = cv2.imread(img, 0)
  train_lab.append(cv2.resize(m, dim).reshape(-1))
print(len(train_lab), train_lab[0].shape)

In [ ]:
len(train_lab)
train_labels = np.array(train_lab)
train_labels.shape

In [ ]:
train_images = np.array(train_img)
train_labels = np.array(train_lab)
print(train_images.shape, train_labels.shape)

In [ ]:
display_img(train_lab[0].reshape(dim))

In [ ]:
#train_images = np.array(train_img)
#train_labels = np.array(train_lab)
print(train_images.shape, train_labels.shape)
print(int(train_images.shape[0]*.7))

In [ ]:
dim = (200,200)

In [ ]:
#create a data frame with test data image - using a numpy array
def create_img_df(img_array, dim):
  df_image = pd.DataFrame()
  train_img = []

for img in train_images:
  m = cv2.imread(img, 0)
  train_img.append(cv2.resize(m, dim).reshape(-1))
print(len(train_img), train_img[0].shape)

  df_image = pd.DataFrame() 
  for img in img_array[:100]:
    df1 = pd.DataFrame()
    #img = cv2.imread(image, 0)
    df1['gray']= img/255
    #df1['Image_Name'] = image.split('/')[-1]

    _,thresh2  = cv2.threshold(img, 80, 255, cv2.THRESH_BINARY_INV)
    thresh2 = thresh2/255
    df1['binay_inv']= thresh2.reshape(-1)
    df1['adaptive_thresh']=(cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11, 11)/255).reshape(-1)
    df1['gabor1'] = gabor_feature_extraction(img,.8, .05 ).reshape(-1)
    df1['gabor2'] = gabor_feature_extraction(img,1.6, .5 ).reshape(-1)
    df1['blur'], df1['laplacian'], df1['sobelx'], df1['canny'] = preprocess_image(img)
    df_image = pd.concat([df_image, df1])
    return df_image
train_image = create_img_df(train_images)
test_image = create_img_df(train_images)
print(df_image.shape)

In [ ]:
#Creating test mask image Data Frame
%%time
df_mask = pd.DataFrame() 
for mask in train_labels[:100]:
  df2 = pd.DataFrame()
  #msk = cv2.imread(mask, 0)
  df2['label'] = mask #msk.reshape(-1) 
  #df2['Mask_Name'] = mask.split('/')[-1]  
  df_mask = pd.concat([df_mask, df2])
print(df_mask.shape)

In [ ]:
df_image.head()

In [ ]:
df_mask.head()

In [ ]:
df = pd.concat([df_image, df_mask], axis=1)
print(df.shape)
df.head()

## 1

In [ ]:
#Now, add a column in the data frame for the Labels
#For this, we need to import the labeled image
labeled_img = cv2.imread(lab)
#Remember that you can load an image with partial labels 
#But, drop the rows with unlabeled data

labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_BGR2GRAY)
labeled_img1 = labeled_img.reshape(-1)


In [ ]:
labeled_img1.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
Y = LabelEncoder().fit_transform(labeled_img1)

In [ ]:
Y.shape

In [ ]:
)

In [ ]:
original_img_data = df.drop(labels = ["Labels"], axis=1) #Use for prediction
#df.to_csv("Gabor.csv")
df = df[df.Labels != 0]

In [ ]:
#Define the dependent variable that needs to be predicted (labels)
Y = df["Labels"].values

#Encode Y values to 0, 1, 2, 3, .... (NOt necessary but makes it easy to use other tools like ROC plots)
from sklearn.preprocessing import LabelEncoder
Y = LabelEncoder().fit_transform(Y)


#Define the independent variables
X = df.drop(labels = ["Labels"], axis=1) 

In [ ]:
#Split data into train and test to verify accuracy after fitting the model. 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=20)



In [ ]:
features.shape, Y.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with n number of decision trees
model = RandomForestClassifier(n_estimators = 20, random_state = 42)

# Train the model on training data
model.fit(features, Y)

In [ ]:
feature_list = list(X.columns)
feature_imp = pd.Series(model.feature_importances_,index=feature_list).sort_values(ascending=False)
print(feature_imp)

In [ ]:
prediction_RF = model.predict(features)

In [ ]:
prediction_RF.shape

In [ ]:
plt.imshow(prediction_RF.reshape(gray.shape))

In [ ]:
lab = cv2.imread('/content/content/processed_labels/aachen_000159_000019.png',0)
print(lab.shape)

In [ ]:
plt.imshow(lab)

In [ ]:
img1 = train_images[11]
lab1 = train_labels[11]
lab1 = cv2.imread(lab1,0)
img1 = cv2.imread(img1,0)
print(lab1.shape, img1.shape)

In [ ]:
plt.imshow(img1)

In [ ]:
plt.imshow(lab1)

Evl

In [ ]:
gabor1 = gabor_feature_extraction(img1,.8, .05 )
gabor2 = gabor_feature_extraction(img1,1.6, .5 )
blur1, blur2, sobele, canny = preprocess_image(img1)
print(blur1.shape, blur2.shape, sobele.shape, canny.shape )
print(gabor1.shape, gabor2.shape)

In [ ]:
features = np.hstack((gray1, gabor1, gabor2, blur1, blur2, sobele, canny))
features.shape

In [ ]:
lab1.shape

In [ ]:
prediction_RF = model.predict(features)

In [ ]:
plt.imshow(prediction_RF.reshape(gray.shape))

In [ ]:
plt.imshow(lab1)

In [ ]:
from sklearn import metrics
#Print the prediction accuracy
#Check accuracy on test dataset. If this is too low compared to train it indicates overfitting on training data.
print ("Accuracy using Random Forest= ", metrics.accuracy_score(y_test, prediction_RF))

In [ ]:
i = load_img('/content/aachen_000000_000019.png')
display_img(i)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
/content/content/processed_images/dusseldorf_000143_000019.png
/content/content/processed_labels/dusseldorf_000143_000019.png

In [ ]:
img, lab

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier



In [ ]:
def preprocess_image(img):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply Canny edge detection
    edges = cv2.Canny(gray, 100, 200)
    
    # Apply Gaussian blur to reduce noise
    blur = cv2.GaussianBlur(edges, (5, 5), 0)
    
    return blur

In [ ]:
def npreprocess_image(img):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).reshape(-1,1)
    sobels = sobel(gray)
    
    # Apply Canny edge detection
    edges = cv2.Canny(gray, 100, 200)
    
    # Apply Gaussian blur to reduce noise
    blur = cv2.GaussianBlur(edges, (5, 5), 0)
    
    return np.hstack((gray, sobels,edges,blur))

In [ ]:
def extract_features(img):
    # Extract RGB color features
    r, g, b = cv2.split(img)
    features = np.dstack((r, g, b)).reshape(-1, 3)
    
    # Extract texture features using Gabor filter
    kernel = cv2.getGaborKernel((5, 5), 4, np.pi/4, 8, 0.5, 0, ktype=cv2.CV_32F)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    filtered = cv2.filter2D(gray, cv2.CV_8UC3, kernel)
    texture_features = filtered.reshape(-1, 1)
    
    # Combine color and texture features
    features = np.hstack((features, texture_features))
    
    return features

In [ ]:
def segment_image(img):
    # Preprocess the image
    preprocessed = preprocess_image(img)
    
    # Extract features from the preprocessed image
    features = extract_features(img)
    
    # Cluster the features using k-means
    kmeans = KMeans(n_clusters=3, random_state=0).fit(features)
    labels = kmeans.labels_.reshape(preprocessed.shape[:2])
    
    # Train a Random Forest classifier on the labeled features
    flat_features = features.reshape(-1, features.shape[-1])
    clf = RandomForestClassifier(n_estimators=50, random_state=0)
    clf.fit(flat_features, labels.ravel())
    
    # Predict the segmentation for the preprocessed image
    flat_preprocessed = preprocessed.reshape(-1, preprocessed.shape[-1])
    segmentation = clf.predict(flat_preprocessed).reshape(preprocessed.shape[:2])
    
    # Convert the segmentation to a binary mask
    mask = np.zeros_like(segmentation)
    mask[segmentation == 1] = 255
    
    return mask

In [ ]:
# Load an example image
img = cv2.imread('/content/content/processed_images/dusseldorf_000143_000019.png')

In [ ]:

# Preprocess the image
preprocessed = preprocess_image(img)
npreprocessed = npreprocess_image(img)
print(preprocessed.shape, npreprocessed.shape)

In [ ]:
# Extract features from the preprocessed image
features = extract_features(img)
features.shape

In [ ]:
# Cluster the features using k-means
kmeans = KMeans(n_clusters=21, random_state=0).fit(features)
labels = kmeans.labels_.reshape(preprocessed.shape[:2])
labels.shape

In [ ]:
preprocessed.shape[:2]

In [ ]:
# Cluster the features using k-means
kmeans = KMeans(n_clusters=21, random_state=0).fit(features)
labels = kmeans.labels_.reshape(npreprocessed.shape[:2])
labels.shape

In [ ]:
labels[10]

In [ ]:
flat_features = features.reshape(-1, features.shape[-1])
print(flat_features.shape)

In [ ]:
clf = RandomForestClassifier(n_estimators=50, random_state=0)

In [ ]:
labels.ravel().shape

In [ ]:
clf.fit(flat_features, labels.ravel())

flat_features = features.reshape(-1, features.shape[-1])
print(flat_features.shape)

In [ ]:
# Predict the segmentation for the preprocessed image
#flat_preprocessed = preprocessed.reshape(-1, preprocessed.shape[-1])
flat_preprocessed = preprocessed.reshape(-1, preprocessed.shape[-1])
flat_preprocessed.shape

In [ ]:
#segmentation = clf.predict(flat_preprocessed).reshape(preprocessed.shape[:2])   
segmentation = clf.predict(npreprocessed)

segmentation.shape

In [ ]:
segmentation = segmentation.reshape(preprocessed.shape[:2]) 
segmentation.shape

In [ ]:
plt.imshow(segmentation)

In [ ]:
# Convert the segmentation to a binary mask
mask = np.zeros_like(segmentation)
mask[segmentation == 1] = 255

In [ ]:


segmentation = clf.predict(flat_preprocessed).reshape(preprocessed.shape[:2])

# Convert the segmentation to a binary mask
mask = np.zeros_like(segmentation)
mask[segmentation == 1] = 255


In [ ]:
# Segment the image
##mask = segment_image(img)



In [ ]:
mask.shape, img.shape

In [ ]:
plt.imshow(img)

In [ ]:
plt.imshow(mask)

In [ ]:
# Display the results
cv2.imshow('Input Image', img)
cv2.imshow('Segmentation Mask', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def preprocess_image(img):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).reshape(-1,1)
    sobels = sobel(gray)
    
    # Apply Canny edge detection
    edges = cv2.Canny(gray, 100, 200)
    
    # Apply Gaussian blur to reduce noise
    blur = cv2.GaussianBlur(edges, (5, 5), 0)
    
    return np.hstack((gray, sobels,edges,blur))

In [ ]:
a = preprocess_image(img)


In [ ]:
a.shape

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray.shape
g= gray.reshape(-1, 1)
g.shape

In [ ]:
#SOBEL
s = sobel(g)
#s = s.reshape(-1,1)
s.shape

In [ ]:
 # Apply Canny edge detection
edges = cv2.Canny(gray, 100, 200)
edges.shape
e= edges.reshape(-1,1)

In [ ]:
blur = cv2.GaussianBlur(edges, (5, 5), 0)
blur.shape
b= blur.reshape(-1,1)

In [ ]:
blur.reshape(-1).shape

In [ ]:
blur.shape

In [ ]:
f = np.hstack((g, e,b,s))
f.shape

In [ ]:
segmentation = clf.predict(f).reshape(preprocessed.shape[:2])
    

In [ ]:
mask = np.zeros_like(segmentation)
mask[segmentation == 1] = 255

In [ ]:
plt.imshow(mask)

In [ ]:
lab = train_label[0]
lab = cv2.imread(lab, 0)


In [ ]:
plt.imshow(lab)


In [ ]:
    segmentation = clf.predict(flat_preprocessed).reshape(preprocessed.shape[:2])
    
    # Convert the segmentation to a binary mask
    mask = np.zeros_like(segmentation)
    mask[segmentation == 1] = 255
    
    return mask

# Load an example image
img = cv2.imread('example.jpg')

# Segment the image
mask = segment_image(img)

# Display the results
cv2.imshow('Input Image', img)
cv2.imshow('Segmentation Mask', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 3

In [ ]:
features = np.hstack((gray1, gabor1, gabor2, blur1, blur2, sobele, canny))
features.shape

In [ ]:
#create a data frame with test data image - using a numpy array
%%time
a = []
for img in train_images[:10]:
  m = img.reshape(-1,1)

  _,thresh2  = cv2.threshold(img, 80, 255, cv2.THRESH_BINARY_INV)
  thresh2 = thresh2.reshape(-1,1)

  adaptive_thresh=(cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11, 11)/255).reshape(-1,1)
  f = np.hstack((m, thresh2, adaptive_thresh))
  a.append(f)


In [ ]:
len(a)

In [ ]:
b = np.array(a)

In [ ]:
b.shape

In [ ]:
#Import training classifier
from sklearn.ensemble import RandomForestClassifier
## Instantiate model with n number of decision trees
rf = RandomForestClassifier(n_estimators = 50, random_state = 42)

In [ ]:
model.fit(X, Y)

In [ ]:
a = load_img('/content/content/processed_labels/aachen_000000_000019.png')

In [ ]:
display_img(a)

In [ ]:
plt.imshow(a)